In [1]:
#%%
## Import Library
import copy
import openai
from azure.core.credentials import AzureKeyCredential
from azure.identity import AzureDeveloperCliCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    HnswParameters,
    PrioritizedFields,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticSettings,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
)
from azure.storage.blob import BlobServiceClient
from langchain.chains import LLMChain
from langchain.llms import AzureOpenAI 
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import CosmosDBChatMessageHistory
import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError
import json

from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
#from langchain.retrievers import AzureCognitiveSearchRetriever
from langdetect import detect
from langchain.prompts import PromptTemplate
import re
# Create chain to answer questions
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import ConversationalRetrievalChain

# Import Azure OpenAI
from langchain.llms import AzureOpenAI 
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

import openai
from azure.core.credentials import AzureKeyCredential
from azure.identity import AzureDeveloperCliCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    HnswParameters,
    PrioritizedFields,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticSettings,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
)

from azure.storage.blob import BlobServiceClient
from pypdf import PdfReader
from langchain.schema import Document
import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
#import textwrap
import logging
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError
import json
from docx import Document as DocxDocument
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import re

In [25]:
index_name = "credit-proposal"
search_service = "gptdemosearch"
search_api_key = "PcAZcXbX2hJsxMYExc2SnkMFO0D94p7Zw3Qzeu5WjYAzSeDMuR5O"
storage_service = "creditproposal"
storage_api_key = "hJ2qb//J1I1KmVeDHBpwEpnwluoJzm+b6puc5h7k+dnDSFQ0oxuh1qBz+qPB/ZT7gZvGufwRbUrN+ASto6JOCw=="
connect_str = f"DefaultEndpointsProtocol=https;AccountName={storage_service};AccountKey={storage_api_key}"

doc_intell_endpoint = "https://doc-intelligence-test.cognitiveservices.azure.com/"
doc_intell_key = "9fac3bb92b3c4ef292c20df9641c7374"

# set up openai environment
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://pwcjay.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-09-01-preview"
os.environ["OPENAI_API_KEY"] = "f282a661571f45a0bdfdcd295ac808e7"

os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = search_service
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = search_api_key
os.environ["AZURE_INDEX_NAME"] = index_name

In [26]:
# set up this varible to mock as front end return sector name
section = 'Executive Summary'

In [27]:
rm_note_txt = """Client: GogoX
Industry: Logistics and Delivery Services
Date: 20 Nov 2023


1	Company Background:		
	GogoX is a leading logistics and delivery service provider, offering on-demand delivery solutions to individuals and businesses. Established in 2013, the company has rapidly expanded its operations and established a strong presence in the market. GogoX operates a user-friendly mobile application and web platform, connecting customers with a network of professional drivers and delivery partners.		

2	Financial Performance:		
GogoX has demonstrated consistent revenue growth over the past few years, driven by increasing customer adoption and expansion into new markets.			
The company's financial statements reflect a healthy profitability margin, indicating effective cost management and operational efficiency.			
Cash flow from operations has been positive, providing a stable source of funds to support working capital requirements and ongoing business operations.			

3	Market Position and Competitive Landscape:		
GogoX has successfully positioned itself as a market leader in the logistics and delivery industry, leveraging its strong brand recognition and innovative technology platform.			
The company has built a robust network of drivers and delivery partners, enabling quick and reliable service fulfillment.			
GogoX's competitive advantage lies in its ability to offer cost-effective and flexible solutions tailored to meet the needs of various customer segments, including e-commerce, retail, and individual users.			

4	Growth Strategy and Market Potential:		
GogoX has outlined a comprehensive growth strategy focused on expanding its geographical presence, diversifying its service offerings, and enhancing customer experience.			
The company plans to enter new markets, both domestically and internationally, to capture additional customer segments and increase market share.			
GogoX aims to invest in technology and infrastructure improvements to streamline operations, optimize delivery routes, and enhance overall efficiency.			

5	Risk Assessment:		
The logistics industry is subject to various risks, including intense competition, regulatory changes, and economic downturns. GogoX has implemented risk mitigation measures such as diversification of services and markets, maintaining strong relationships with key partners, and closely monitoring market trends.			
Operational risks, such as driver availability, vehicle maintenance, and service disruptions, are managed through rigorous driver screening, continuous training programs, and proactive maintenance schedules.			
Financial risks are mitigated by maintaining a healthy liquidity position, diversifying funding sources, and prudent financial management practices.			

6	Credit Request and Repayment Plan:		
GogoX is requesting a credit facility of $10million to support its expansion plans, technology investments, and working capital needs.			
The proposed repayment plan consists of regular principal and interest payments over a 3 years term, aligning with the company's projected cash flow generation and financial performance.			

7	Shareholders and Group Structure:		
GogoX is a privately-held company, no detailed shareholder information			
Key investors of GOGOX include New Horizon Capital, Alibaba Entrepreneurs Fund, and InnoVision Capital			
			
Conclusion:			
GogoX has demonstrated a strong market position, consistent financial performance, and a well-defined growth strategy. With its robust operational capabilities, innovative technology platform, and customer-centric approach, the company is well-positioned to capitalize on the growing demand for logistics and delivery services. The proposed credit facility, in line with the company's financial projections, will support GogoX's expansion plans and enable it to maintain its competitive edge in the market.			
"""

In [32]:
# Core LLM call funcition

def cap(match):
    return(match.group().capitalize())


def load_json(json_path):
    with open(json_path, "r" ,encoding="utf-8") as f:
        data = json.load(f)
    return data

'''
    You could follow the below extraction example format to output the answer.
    example (Keyword: example):
    {example}

    3. The example (Keyword: proposal_example) above is just for your reference only to improve your theme, you must not directly copy the content in the examples

'''

#This funcition is to prepare the rm note in desired format for web, call by app.py
def web_extract_RM(section, rm_note_txt):
    hierarchy_file_name = "config/hierarchy_v2.json"

    hierarchy_dict_list = load_json(hierarchy_file_name)["content"]
    print(hierarchy_dict_list)
    prompt_template_for_extracting_rm_note = """
    Read the following context, aggregate the context and answer the input question based on the aggregate context (Keyword: Question):
    
    Please Just based on the 
    context: {rm_note}

    ======
    Question: {question}
    ======

    Follow the instruction below:
    1. Please provide your answer in English
    2. Do not start your answer with "Based on the given information"
    3. If possible, try to expand the information provided from the RM
    4. Do not create any figures by make-up 
    5. Please provide [N/A] as answer if you cannot find any relevant information from the given context. Example Format: [N/A]
    
    Take a deep breath and work on this step by step
    """
    rm_prompt_template = PromptTemplate(template=prompt_template_for_extracting_rm_note, input_variables=["rm_note", "question",])# "example",])

    llm_rm_note = AzureChatOpenAI(deployment_name="gpt-35-16k", temperature=0.1,
                            openai_api_version="2023-05-15", openai_api_base="https://pwcjay.openai.azure.com/")

    #"example": dictionary["Example"],

    output_dict_list = []
    for dictionary in hierarchy_dict_list:
        print(dictionary)
        if dictionary["Section"] == section:
            chain = LLMChain(llm=llm_rm_note, prompt=rm_prompt_template)
            dictionary["Value"] = chain({"rm_note":rm_note_txt, "question": dictionary["Question"]})['text']
            dictionary["Value"] = dictionary["Value"].replace("Based on the given information, ", "")
            if "[N/A]" in dictionary["Value"]:
                dictionary["Value"] = ""
            output_dict_list.append(dictionary)

    # Create Json file 
    # output_json_name = "GOGOVAN_hierarchy_rm_note.json"
    # json.dump(output_dict_list, open(output_json_name, "w"), indent=4)

    return output_dict_list

'''
        ======
        Example: (Keyword: proposal_example)
        {example}
        ======

'''


def first_gen_template():
    proposal_proposal_template_text = """
        Read the input json for this section carefully and aggregate the content of "Value" key:

        Please follow the format in content of "example" key to output the answer
        But please do not include any content in "example" key to output the answer, as it just use as reference


        Read the input json for this section carefully and aggregate the content of "Value" key
        please based on the content in "Value" key to output the answer
        Input JSON:
        {input_json}
        
        Then write paragraph(s) based on the above aggregrated context 

        Rules you need to follow:
        1. Don't mention the word "RM Note" and "Component", and don't mention you held a meeting with the client! Instead, you shall say "It is mentioned that"
        2. Don't mention the source of your input (i.e. RM Note (Keyword: rm_note), example, document)
        3. Don't justify your answers
        4. Don't provide suggestion or recommendation by yourself
        5. Provide your answer in English
        6. Breake it to multi-paragraphs if one single paragraph consists of more than 100 words
        7. In the same paragraph, don't input line breaks among the sentences
        8. Don't start with your answer by a title. You must start your paragraph immediately
        9. The example (Keyword: proposal_example) above is just for your reference only to improve your theme, you must not directly copy the content in the examples
        10. If possible, you can use point-form, tables to provide your answer
        11. Don't introduce what this section includes

        Guidance when you do not have the information:
        1. When you don't have the specific information or you need further information (Keyword: further_info), you have to write it in the following format: [RM please helps provide the further information of (Keyword: further_info)], where please supplement the information you need here.
        2. You must not create the information by yourself if you don't have relevant information
        3. You cannot say "It's unclear that", please refer to point 1 for the formatting for requesting further information

        Take a deep breath and work on this step by step
        """
    prompt_template_proposal = PromptTemplate(template=proposal_proposal_template_text, input_variables=["input_json"])


    return prompt_template_proposal


def regen_template():
    proposal_proposal_template_text = """
        Read the previous_paragraph, RM instruction for this section carefully:
        
        Please follow the previous paragraph and RM instruction and return a summarize paragraph in human readable form:
        {previous_paragraph}

        ======================================

        Please follow the RM instruction to edit the previous_paragraph and return a summarize paragraph in human readable form:
        You could treat the RM instruction as prompt.
        RM instruction:
        {rm_instruction}
        ======================================

        Then write paragraph(s) based on the above aggregrated context

        Rules you need to follow:
        1. Don't mention the word "RM Note" and "Component", and don't mention you held a meeting with the client! Instead, you shall say "It is mentioned that"
        2. Don't mention the source of your input (i.e. RM Note (Keyword: rm_note), example, document)
        3. Don't justify your answers
        4. Don't provide suggestion or recommendation by yourself
        5. Provide your answer in English
        6. Breake it to multi-paragraphs if one single paragraph consists of more than 100 words
        7. In the same paragraph, don't input line breaks among the sentences
        8. Don't start with your answer by a title. You must start your paragraph immediately
        9. The example (Keyword: proposal_example) above is just for your reference only to improve your theme, you must not directly copy the content in the examples
        10. If possible, you can use point-form, tables to provide your answer
        11. Don't introduce what this section includes

        Guidance when you do not have the information:
        1. When you don't have the specific information or you need further information (Keyword: further_info), you have to write it in the following format: [RM please helps provide the further information of (Keyword: further_info)], where please supplement the information you need here.
        2. You must not create the information by yourself if you don't have relevant information
        3. You cannot say "It's unclear that", please refer to point 1 for the formatting for requesting further information

        Take a deep breath and work on this step by step
        """
    prompt_template_proposal = PromptTemplate(template=proposal_proposal_template_text, input_variables=["previous_paragraph", "rm_instruction"])


    return prompt_template_proposal


def clean_generated_text(text, section_name, client):
    #replacement
    text = text.replace("Based on the given information, ", "").replace("It is mentioned that ", "")
    
    #reformat the client name
    insensitive_replace = re.compile(re.escape(client.lower()), re.IGNORECASE)
    text = insensitive_replace.sub(client, text)

    #Drop some unwanted sentences
    sentence_list = re.split(r"(?<=[.?!])", text)
    unwanted_word_list = ["ABC ", "XYZ ", "GHI", "DEF "]
    sentence_list_dropped = [sentence.strip() for sentence in sentence_list if all(word not in sentence for word in unwanted_word_list)]
    text = ' '.join(sentence_list_dropped)

    #Remove the section name if it starts with it
    if text.lower().startswith(section_name.lower()+": "):
        text = text[len(section_name)+2:]

    #All capital letters for first letter in sentences
    formatter = re.compile(r'(?<=[\.\?!]\s)(\w+)')
    text = formatter.sub(cap, text)
    text = text[0].upper()+text[1:]
    return text

def generate_rm_fill(gpt_generated_text, client):
    rm_fill_values = []
    additional_info_values = []
    lines = gpt_generated_text.split("\n")
    for line in lines:
        match = re.search(r"\[RM .+?\]", line)
        if match:
            rm_fill = match.group(0)
            # Remove the '[RM ' at the start and ']' at the end, then append
            rm_fill = rm_fill.replace('[RM ', '', 1)
            rm_fill = rm_fill[:-1] # remove the closing bracket
            rm_fill_values.append(rm_fill)
            line = line.replace(rm_fill, "")
                
        # Check for "Please provide further information" in the line
        if "Please provide further information" in line:
            # Append the whole line
            additional_info_values.append(line)

    # Join all the strings in the list with a space in between each string
    rm_fill_text = ' '.join(rm_fill_values)
    additional_info_text = ' '.join(additional_info_values)

    # Combine rm_fill_text and additional_info_text into one string
    combined_text = rm_fill_text + ' ' + additional_info_text

    #reformat the client name
    insensitive_replace = re.compile(re.escape(client.lower()), re.IGNORECASE)
    combined_text = insensitive_replace.sub(client, combined_text)
    return combined_text

# to first generate 
def first_generate(section_name, input_json, client):
    """
    A core function to generate the proposal per section

    Parameters:
    -----------
    prompt: str
    Prompt text for instructing the output based on RM prompt

    rm_note: str
    It contains the information from the RM

    example: str
    Input example for GPt to take it as an example

    """
    #.format(content=content, section=section, context=context)
    
    prompt_template_proposal = first_gen_template()

    llm_proposal = AzureChatOpenAI(deployment_name="gpt-35-16k", temperature=0,
                            openai_api_version="2023-05-15", openai_api_base="https://pwcjay.openai.azure.com/")
    
    chain = LLMChain(
        llm=llm_proposal,
        prompt=prompt_template_proposal
    )

    drafted_text = chain({"input_json": input_json
                    ,})['text']
    print(drafted_text)
    drafted_text2 = clean_generated_text(drafted_text, client, section_name)
    combined_text = generate_rm_fill(drafted_text2, client)
    output_json = {
        "section": section_name,
        "output": drafted_text2,
        "RM fill" : combined_text,
    }
    #output the result
    return output_json

def run_first_gen(section, rm_note_txt, client):

    extract_json = web_extract_RM(section ,rm_note_txt)
    output_json = first_generate(section, extract_json, client)

    return output_json

def regen(section_name, previous_paragraph, rm_instruction):
    prompt_template_proposal = regen_template()

    llm_proposal = AzureChatOpenAI(deployment_name="gpt-35-16k", temperature=0,
                            openai_api_version="2023-05-15", openai_api_base="https://pwcjay.openai.azure.com/")
    
    chain = LLMChain(
        llm=llm_proposal,
        prompt=prompt_template_proposal
    )

    drafted_text = chain({"previous_paragraph": previous_paragraph
                    ,"rm_instruction":rm_instruction})['text']
    
    output_json = {
        "section": section_name,
        "output": drafted_text2,
        "RM fill" : combined_text,
    }
    #output the result
    return output_json

def create_docx(client_name, json_data):
    # Create a new Word document
    document = DocxDocument()

    title_text = "Credit Proposal for " + client_name
    title_size = 20 # Font size in points

    # Create a paragraph for the title
    title_paragraph = document.add_paragraph()

    # Add the title text to the paragraph
    title_run = title_paragraph.add_run(title_text)

    # Apply formatting to the title run
    title_run.bold = True
    title_run.font.size = Pt(title_size)

    # Set the alignment of the paragraph to the center
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Convert JSON values to section headers and paragraphs in the Word document
    for item in json_data['consolidated_text']:
        section = item['section']
        context = item['output']

        # Add the section header
        document.add_heading(section, level=1)

        # Split context into lines and check each line
        for line in context.split('\n'):
            # Create a new paragraph for each line
            paragraph = document.add_paragraph()

            # Search for the pattern [RM please ... ] using regex
            matches = re.findall(r'\[RM .*?\]', line)

            if matches:
                # If there's a match, split line into parts
                parts = re.split(r'(\[RM .*?\])', line)

                for part in parts:
                    run = paragraph.add_run(part)

                    if part in matches:
                        # This part should be colored red
                        run.font.color.rgb = RGBColor(255, 0, 0)  # RGB values for red
            else:
                # Normal text
                run = paragraph.add_run(line)

    # Save the Word document
    document.save(client_name + '_Word_proposal.docx')


In [33]:
run_first_gen(section, rm_note_txt, "GogoX")

[{'Section': 'Executive Summary', 'Sub-section': 'EXECUTIVE SUMMARY', 'Breakdown': '', 'Value': '', 'Question': 'Please extract the EXECUTIVE SUMMARY', 'Example': 'The deal is referred by Pamfleet Group (“Pamfleet”) and our relationship with it can be traced back to early 2006 when the OIC was with Hang Seng Bank.  After joining ICBC (Asia), OIC’d tried a couple of time to cooperate with Pamfleet, but was in vain owing to the more aggressive offer given by Pamfleet’s partner banks, like CITIC Ka Wah Bank.  However, due to ICBCA business scale, network plus OIC’s marketing effort and cordial relationship, Pamfleet agreed to provide an industry / office acquisition project for our consideration. In this project, Pamfleet is to cooperate with Angelo Gordon Group (“Angelo Gordon”), a US investment company dedicated to alternative investment to form a fund, which will be used to acquire Ever Gain Plaza located in Kwai Chung (the “Property”) or the companies holding the Property.', 'ID': 0},

C:\Users\Jay KK Hui\AppData\Roaming\Python\Python39\site-packages\langchain\chat_models\azure_openai.py:155: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://pwcjay.openai.azure.com/ to https://pwcjay.openai.azure.com/openai.
  warnings.warn(
C:\Users\Jay KK Hui\AppData\Roaming\Python\Python39\site-packages\langchain\chat_models\azure_openai.py:162: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
C:\Users\Jay KK Hui\AppData\Roaming\Python\Python39\site-packages\langchain\chat_models\azure_openai.py:170: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resour

{'Section': 'Executive Summary', 'Sub-section': 'Overview of the Proposal', 'Breakdown': '', 'Value': '', 'Question': 'Please extract the Overview of the Proposal', 'Example': 'Introduction of the deal brought by Pamfleet Group and the aim to form a fund with Angelo Gordon Group for the acquisition of Ever Gain Plaza or the companies holding the Property.', 'ID': 1}
{'Section': 'Client Request', 'Sub-section': 'Purpose of the Loan', 'Breakdown': '', 'Value': '', 'Question': 'Please extract the Purpose of the Loan', 'Example': 'Acquisition of the majority of Ever Gain Plaza by Pamfleet and Angelo Gordon.', 'ID': 2}
{'Section': 'Client Request', 'Sub-section': 'Amount Requested', 'Breakdown': '', 'Value': '', 'Question': 'Please extract the Amount Requested', 'Example': 'Term Loan Facility of HKD757.5M for acquisition. IRS Facility of HKD757.5M (notional) for hedging the interest rate risk of the Facility.', 'ID': 3}
{'Section': 'Client Request', 'Sub-section': 'Terms and Conditions Requ

C:\Users\Jay KK Hui\AppData\Roaming\Python\Python39\site-packages\langchain\chat_models\azure_openai.py:155: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://pwcjay.openai.azure.com/ to https://pwcjay.openai.azure.com/openai.
  warnings.warn(
C:\Users\Jay KK Hui\AppData\Roaming\Python\Python39\site-packages\langchain\chat_models\azure_openai.py:162: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
C:\Users\Jay KK Hui\AppData\Roaming\Python\Python39\site-packages\langchain\chat_models\azure_openai.py:170: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resour

It is mentioned that GogoX is a leading logistics and delivery service provider in the industry. The company was established in 2013 and has rapidly expanded its operations, becoming a market leader. GogoX operates a user-friendly mobile application and web platform, connecting customers with professional drivers and delivery partners. The company has demonstrated consistent revenue growth, indicating effective cost management and operational efficiency. Cash flow from operations has been positive, providing a stable source of funds.

GogoX has positioned itself as a market leader by leveraging its strong brand recognition and innovative technology platform. It has built a robust network of drivers and delivery partners, offering cost-effective and flexible solutions to various customer segments. The company has outlined a growth strategy focused on expanding its geographical presence, diversifying its service offerings, and enhancing customer experience. GogoX plans to enter new marke

{'section': 'Executive Summary',
 'output': "GogoX is a leading logistics and delivery service provider in the industry. The company was established in 2013 and has rapidly expanded its operations, becoming a market leader. Gogox operates a user-friendly mobile application and web platform, connecting customers with professional drivers and delivery partners. The company has demonstrated consistent revenue growth, indicating effective cost management and operational efficiency. Cash flow from operations has been positive, providing a stable source of funds. Gogox has positioned itself as a market leader by leveraging its strong brand recognition and innovative technology platform. It has built a robust network of drivers and delivery partners, offering cost-effective and flexible solutions to various customer segments. The company has outlined a growth strategy focused on expanding its geographical presence, diversifying its service offerings, and enhancing customer experience. Gogox p

In [31]:
previous_paragraph = run_first_gen(section, rm_note_txt)
previous_paragraph['output']
regen(section, previous_paragraph['output'], "Gogovan is requesting a credit facility of $80 million instead of 10 million")

{'section': 'Executive Summary',
 'output': 'Gogovan is seeking a credit facility of $80 million instead of $10 million to support their expansion plans, technology investments, and working capital needs. The proposed repayment plan still consists of regular principal and interest payments over a 3-year term. \n\nGogovan aims to expand its geographical presence, diversify its service offerings, and enhance customer experience with the increased credit facility. However, specific details regarding their expansion plans, technology investments, and working capital needs are not provided in the given information. \n\n[RM please help provide further information on the specific expansion plans, technology investments, and working capital needs of Gogovan.]',
 'RM fill': 'please help provide further information on the specific expansion plans, technology investments, and working capital needs of Gogovan. '}

In [39]:
# Usage
json_data = {
    "consolidated_text":[
        {
            "section":"xxx",
            "output":"xxx"
        },
        {
            "section":"yyy",
            "output":"yyy"
        },
    ]
}

create_docx('GOGOX', json_data)